# 9 - Model Validation

Once we've created a model for our data set, how do we test how "good" our model is?  There are a number of statistical tools we can use to determine this, with different options for different kinds of models.  Before we look at specific tools, we'll go over some more general statistical ideas about testing hypotheses.

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (12.0, 10.0)

Populating the interactive namespace from numpy and matplotlib


## Hypothesis Testing

### Null Hypothesis

Given a data set, one often wants to test if there is an intrinsic relationship between subsets of the data, or to use the data to attempt to validate a theory.  One statistical approach to accomplishing this is the use of a **null hypothesis**, often denoted $H_0$. The idea is similar to "proof by contradiction" in mathematics: 
1. Denote the hypothesis we wish to validate $H_1$, the **alternative hypothesis**.
2. Suppose that $H_0$ is true: there is no intrinsic relationship in the data, that the results occurred purely by chance.
3. Using a statistical test, compute if $H_0$ is incredibly unlikely.  
4. If so, reject $H_0$.  Else, reject $H_1$.

In the case where we do not reject the null hypothesis, we would have to reformulate our initial hypothesis for future testing (or discard it entirely).  

As an example, suppose we had measurements of ancient Egyptian skulls through archaeological finds.  One hypothesis we may wish to test is "the breadth of skulls in humans increases over time", using current average skull breadth measurements as the testing criterion.  The null hypothesis would be that the skull breadths *do not* increase over time, i.e.: the average of the measurements in the sample data would be greater than or equal to the current skull size average.  If we had a statistical test that we could apply to show that the null hypothesis is incredibly unlikely considering the skull data we have, then we could reject the null hypothesis in favour of our hypothesis about increasing skull breadth over time.  

But how do we test and decide what's unlikely or not?  One method is using p-values as metrics for statistical tests.  

### P-Values

Suppose we have an observation (data set), a null hypothesis $H_0$, and an alternative hypothesis $H_1$.  If we presume $H_0$ to be true, then the **p-value** is the probability that we would obtain results which are at least as extreme as the observed data.  If the p-value is very low (i.e.: lower than a specified significance level), this tells us that $H_0$ may not be sufficient to explain the data that we have.  This allows us to reject the null hypothesis and accept the alternative hypothesis.  

Significance levels for p-values are chosen often by convention, but are arbitrary.  Often, the choice of p < 0.05 is used.  There is controversy and misunderstanding surrounding p-values. In particular, it should be stressed that p-values should not  be the sole deciding factor in inference, but should be used in conjunction with context about the data.  Ronald Wasserstein & Nicole Lazar note in their paper "The ASA's Statement on p-Values: Context, Process, and Purpose" (see link below), that:

>Researchers should bring many contextual factors into play to derive scientific inferences, including the design of a study, the quality of the measurements, the external evidence for the phenomenon under study, and the validity of assumptions that underlie the data analysis. Pragmatic considerations often require binary, “yes-no” decisions, but this does not mean that p-values alone can ensure that a decision is correct or incorrect. The widespread use of “statistical significance” (generally interpreted as “p ≤ 0.05”) as a license for making a claim of a scientific finding (or implied truth) leads to considerable distortion of the scientific process.

The key thing to remember is that p-values are computed under the assumption that the null hypothesis is true, and tell you how likely it would be to observe values more extreme than what is in the data set.  As a result, it tells you nothing about the probability of whether either hypothesis is *true* or not.    

Further reading can be found here:

http://ftp.isds.duke.edu/WorkingPapers/03-26.pdf<br>
https://en.wikipedia.org/wiki/Misunderstandings_of_p-values<br>
http://amstat.tandfonline.com/doi/full/10.1080/00031305.2016.1154108

If we've decided to move forward with using p-values, confident in our knowledge of what they are and are not, the question arises:  how do we compute them?  This is generally accomplished by selecting a particular test for the hypothesis, computing a *test statistic*, and using its value with the *sampling distribution* to obtain the p-value.

### Test Statistics

To help us determine whether or not to reject the null hypothesis, we need to choose a statistical test to perform based on the properties of the data, that will quantify a way to distinguish the hypotheses.  Each test specifies a **test statistic**: a single quantity that in some way summarizes important aspects of the data set we're interested in (mean, correlation coefficient, more complicated computations, etc.) which will be used with the observed data to evaluate the likelihood of the null hypothesis.  We can make our method in the null hypothesis section above a little more precise:

1. State the null and alternative hypotheses.
2. Decide on a level of significance to compare the p-value to.
3. Choose a test which suits the kind of data.
4. Compute the value of the test statistic for the data set.
5. Determine the p-value from the test statistic using the sampling distribution.
6. Decide whether or not to reject the null hypothesis by checking the p-value against the level of significance.

There are different categories of tests, depending on the type of data we have, which will change the form of the test statistic.  Two main types are:

* **One-sample tests**: are used when comparing data to a known population / value.  
* **Two-sample tests**: are used when comparing two different data sets. 

We'll have a closer look at a few examples shortly.

### Sampling Distribution

Suppose $x_1,x_2,\ldots,x_n$ are observed values from a data set. We can define the **sample distribution function** $F_n(x)$, in the following way:
1. The argument, $x$, can be any real number.
2. The function outputs the proportion of numbers in our data set that are $\leqslant\, x$. 

i.e.: if, in our data set, there are exactly $k$ values which are less than or equal to x, then $F_n(x) = \frac{k}{n}$.  

For the purposes of statistical testing, we are concerned with computing the sample distribution for the *test statistic* for the test we're performing, given the size of our data set.  This will give us the proportion of values less than or equal to the value we computed, which will let us figure out the p-value (i.e.: the probability of observing something more extreme than what we did).

Actually computing sample distributions can be fairly labour-intensive; before the advent of computer programs capable of computing these numbers on demand, statisticians would have to reference values in very large tables.  We'll be letting Python do the work for us, though.  Let's have a look at different kinds of statistical tests.

### Shapiro–Wilk test

The Shapiro-Wilk test is a way to determine if a given data set is Gaussian.  In the set-up of the test, the null hypothesis is that the given data set **is** normally distributed.  Once we choose a level of significance for our test, we can run the the test and compute both the test statistic and the p-value.  If, for example, we choose our significance level to be 0.05, and the p-value we compute is 0.025 for a given data set $X$, then we reject the null hypothesis that $X$ is normally distributed.  If, however, we compute a p-value of 0.12, then we cannot reject the null hypothesis.

<img src="http://i.imgur.com/HCedwf4.png",width=550,height=550>

For a given data set $X = \{x_1,\ldots,x_n\}$, the Shapiro-Wilk test statistic has the form:

$$ W = \frac{\left(\sum_{i=1}^ka_i(x_{(n-i+1)}-x_{(i)}) \right)^2}{\sum_{i=1}^n(x_i - \mu_X)}, $$

where $x_{(j)}$ denotes the $j^{th}$ element of $X$ when re-arranging into ascending order, $\mu_X$ is the mean of $X$, the coefficients $a_i$ depend on $n$ and are computed via a matrix equation, and $k = \bigl\lfloor \frac{n}{2} \bigr\rfloor$, which is $\frac{n}{2}$ rounded down the nearest integer.

We can use the SciPy function `stats.shapiro(x)` to compute the Shapiro-Wilk test for a given data set `x`.  The function will output two numbers, `(W,p)`, which are the test-statistic and p-value, respectively.  Let's compare two data sets, and choose ahead of time our significance level to be $0.01$:

In [4]:
x1 = [19, 18, 16, 14, 11,  0,  8, 12, 17, 18, 10,  7, 16, 18, 15, 12,  0, 4,  4, 11, 15,  4, 12, 13,  8, 18, 
      11, 18,  3,  5, 18, 14, 12,  3, 15, 18,  2, 18,  2, 18, 12,  7,  2,  7,  2, 17,  9,  9,  4,  6, 69, 97, 
      84, 90, 80, 80, 74, 79, 48, 85, 48, 94, 50, 60, 70, 59, 83, 64, 79, 68, 96, 86, 76, 87, 43, 96, 77, 70, 
      64, 69, 70, 40, 78, 40,  67, 50, 82, 96, 48, 50, 97, 78, 42, 75, 60, 79, 65, 83, 50, 50]
# obtained via concatenating two np.random.randint(a,b,50), one from 0 to 30, and one from 50 to 100.

x2 = [  16.50910448,  22.39398283,  22.27626366,  16.22325598, 21.87551527,  21.40104677,  18.06070508,  17.48751555,
        15.03445503,  19.79045477,  15.3734691 ,  21.06006474, 16.24847507,  25.90433196,  21.00225332,  21.84069789,
        18.16321542,  19.83138415,  14.89816334,  18.22877618, 16.91501954,  17.98657684,  20.45551657,  18.34817168,
        20.85156526,  14.43832657,  23.00033636,  19.07868946, 16.34545215,  19.29270657,  22.65088714,  21.35318661,
        19.76075478,  21.24389901,  15.70178397,  23.09017102, 16.31758063,  17.03078011,  20.64382268,  22.77800154,
        21.67719434,  20.52426192,  12.64210407,  12.19122028, 20.91931133,  17.20447393,  18.35246361,  17.67549573,
        18.63156913,  14.09206448,  19.26118913,  11.50262005, 23.70571152,  17.15269872,  23.01609265,  21.94062937,
        21.01816282,  18.66123268,  18.16277525,  23.70817879, 27.38365385,  21.46336741,  21.42017973,  21.5376097 ,
        15.1677562 ,  19.63875956,  17.87046269,  23.89264974, 26.92608041,  22.40678648,  20.26913699,  19.77450803,
        19.61472674,  21.81003095,  22.02735034,  21.53584979, 19.22672501,  22.23608912,  18.96711669,  17.13995069,
        21.41165054,  16.85484758,  20.87604705,  18.47511586, 14.79669234,  21.72165664,  17.11637749,  25.31159641,
        20.55319247,  16.31414202,  16.14344031,  20.64910554, 17.98645468,  16.30465569,  16.93656026,  18.25318075,
        18.54833335,  15.75328812,  21.74905835,  20.26089833]
# obtained via np.random.normal(loc=20,scale=3,size=100)

In particular, we have good reason to believe that `x2` is normally distributed, since it was generated via a normal random sampling; the set `x1`, however, was generated via uniform random sampling across two different ranges, so there's no particular reason to think this set will be normally distributed.

In [5]:
stats.shapiro(x1)

(0.8689814209938049, 6.37875317011094e-08)

In [6]:
stats.shapiro(x2)

(0.9894847273826599, 0.623079240322113)

1. For `x1`, the p-value is $\cong 0.0000000638$, which is certainly less than our chosen significance level of $0.01$.  Hence, we reject the null hypothesis that `x1` is normally distributed.  
2. For `x2`, the p-value is $\cong 0.623$, which is not less than $0.01$, meaning we cannot reject the null hypothesis that `x2` is normally distributed.

This lines up with our expectations, given the contrived nature of the example.

Testing whether a data set is normally distributed can potentially give us another way to deal with outliers:  the **z-score** of an element $x$ of a data set $X$ with mean $\mu$ and standard deviation $\sigma$ is given by
$$  z = \frac{x-\mu}{\sigma} $$
This is a measure of how many standard deviations from the mean the element $x$ is.  Since $99.7\%$ of data is contained within three standard deviations of the mean for a normally distributed set, this means that if a data point has a z-score of more than three (or less than -3), then we can discard it as an outlier. 

Consider the following data set:

In [7]:
n = [   26.35022317,  20.75227751,  23.47751465,  22.87726524, 22.18087948,  23.8744957 ,  22.84644217,  23.75790925,
        18.10016637,  32.96650641,  26.19575661,  26.06633301, 22.63250235,  28.23314674,  26.92962223,  18.37144332,
        24.28580894,  28.73297662,  19.73941276,  24.18843525, 26.83554143,  23.56251076,  25.14174836,  28.56764978,
        24.35712616,  22.97048562,  26.88372203,  10.21119535, 29.87102246,  22.79502915,  26.94472247,  29.43139378,
        27.72083063,  28.88420645,  24.34621529,  27.13172742, 24.2544851 ,  29.31440951,  23.32003235,  23.40796604,
        27.47217412,  21.95813955,  21.64514648,  25.86996744, 19.56790511,  23.9574127 ,  19.199265  ,  26.52683638,
        27.9121632 ,  27.07325111]

Applying the Shapiro-Wilk test with a significance level of 0.01:

In [8]:
stats.shapiro(n)

(0.9403377175331116, 0.0138937309384346)

The p-value is $\cong 0.014 > 0.01$, so we do not reject the null hypothesis that `n` is normally distributed.  We can compute the z-scores of the elements of `n` with `starts.zscore(n)`:

In [9]:
stats.zscore(n)

array([ 0.46619523, -1.01968619, -0.29631739, -0.4556436 , -0.6404876 ,
       -0.19094539, -0.46382507, -0.22189132, -1.72364492,  2.22237739,
        0.42519465,  0.39084131, -0.52061183,  0.96598592,  0.61998706,
       -1.65163897, -0.08176914,  1.09865744, -1.28853427, -0.10761536,
        0.59501487, -0.27375659,  0.14542572,  1.05477418, -0.06283917,
       -0.4308998 ,  0.60780361, -3.81764085,  1.40073275, -0.4774718 ,
        0.62399516,  1.28404064,  0.83000014,  1.13879888, -0.06573528,
        0.67363251, -0.09008353,  1.25298911, -0.33811845, -0.31477791,
        0.76399842, -0.6996102 , -0.782689  ,  0.33871934, -1.33405812,
       -0.16893645, -1.43190749,  0.51307426,  0.88078619,  0.65811095])

We can find where the absolute z-score is greater than 3:

In [10]:
np.where(abs(stats.zscore(n))>3)

(array([27]),)

In [11]:
n[27]

10.21119535

In [12]:
np.mean(n) - 3*np.std(n)

13.291595308643904

Hence, the element 10.21119535 is less than 3 standard deviations aways from the mean ($\mu -3\sigma \cong 13.29$), and thus we can discard it as an outlier.

### Chi-squared Test

The chi-squared ($\chi^2$) test arises from the chi-squared distribution:

<img src="http://i.imgur.com/B4JWsTU.png",width=600,height=600>

This distribution is related to randomly sampling from a number of normal distributions, where the $k$ values represent the degrees of freedom in the sampling.  Chi-squared tests are specifically for categorical data (i.e.: not numerical, but qualitative data, like names, etc.), and can help determine a number of things, including whether the frequency of observations in a data set diverges from a theoretical distribution of frequencies, and whether two data sets are independent of one another. The chi-squared test statistic has the following form:

$$ Q = \sum_{i=1}^k\frac{(N_i - n{p_i}^0)^2}{n{p_i}^0}  $$

Where $N_i$ is the number of observations in the data that is of type $i$ (for whatever categories of data we have), ${np_i}^0$ is the *expected* number of observations that are of type $i$, and $k$ is the number of different types of data we have in our set.  

Here is an example of a problem we could apply this test statistic to:  AA Publishing prints comic books. It claims that 30% of the comic issues are special variant covers, 65% are normal covers, and 5% are holographic covers.  If we took a random sample of 100 comics that has 50 variant covers, 40 normal covers, and 10 holographic covers, is this consistent with the company's claim? 

To test this, we will use SciPy's `stats.chisquare()` function, which tests the null hypothesis that the observed data has the given expected frequencies.  For problems like the one here, the function takes two arguments:  a collection of sample observations, and the expected observations.  The function outputs two numbers, the chi-squared test statistic value, and the compute p-value.  Let us decide on a significance level of $0.05$ for this test:

In [13]:
AAclaims = [30,65,5]
Observed = [50,40,10]

In [14]:
stats.chisquare(Observed,AAclaims)

Power_divergenceResult(statistic=27.94871794871795, pvalue=8.531256690329268e-07)

So the p-value we computed is $\cong 0.00000212$, which is certainly less than the chosen significance level of $0.05$.  Thus, we must reject the null hypothesis that the observed data has the frequencies claimed by the publishing company.  If we, instead, had randomly sampled 28 variant covers, 63 regular covers, and 9 holographic covers, we'd compute: 

In [15]:
Observed = [28,63,9]
AAclaims = [30,65,5]
stats.chisquare(Observed,AAclaims)

Power_divergenceResult(statistic=3.3948717948717952, pvalue=0.18315254439634107)

The p-value of $\cong 0.18$ is bigger than our significance level of $0.05$, so we could not reject the null hypothesis that the observed data has the frequencies claimed by AA.

### Student's t-Test

The t-test comes from the t-distribution; the red and green graphs show the t-distribution for one and two degrees of freedom, with the blue graph shows the Gaussian distribution for comparison:

<img src="http://i.imgur.com/kA9affe.png",width=550,height=550>


The t-test is used for comparing the mean of data sets:  either comparing the mean of a single data set to a given value, or comparing the means of two data sets.  The (one-sample) t-statistic has the form:

$$ t = \frac{\overline{x} - \mu}{\left(s\,\big/ \sqrt{n}\right)}$$

where $\overline{x}$ is the mean of the sample (the data set we have), $\mu$ is the value we are testing against, $s$ is the standard deviation of the sample, and $n$ is the size of the sample.  There are a number of assumptions underlying the t-test, but one of the major ones is that the populations being compared are both Gaussian.  

We can use the SciPy function `stats.ttest_1samp()` for a one-sample t-test, which takes a set of observations and a given mean as arguments, and outputs the t-statistic value and the p-value.  The null hypothesis being tested is that the mean of a given data set is equal to the specified population mean.

An example of a problem we could apply the test to:  suppose we had a sample of (systolic) blood pressure measurements from 50 first-year EMTs at rest.  Suppose further that we have data for the average resting blood pressure for the general populace. Are the observed data for the EMT blood pressures consistent with the general populace's average?  Let's choose a significance level of $0.05$ for this test:

In [16]:
EMT_bp = [106, 106, 113, 110, 114, 136, 125, 114, 134, 116, 113, 122, 144, 141, 102, 107, 140, 139, 101, 103, 
          114, 113, 109, 133, 139, 137, 122, 121, 108, 110, 135, 111, 115, 105, 112, 109, 139, 119, 133, 132, 
          120, 109, 141, 102, 130, 101, 134, 143, 122, 129]
mean_sbp = 120

In [17]:
stats.ttest_1samp(EMT_bp,mean_sbp)

Ttest_1sampResult(statistic=0.3461821343694726, pvalue=0.7306879364583656)

The p-value for this test was $\cong 0.73$, which is *much* greater than the significance level of $0.05$ we chose.  This means that we cannot reject the null hypothesis, and so can conclude that the systolic blood pressures of the sampled EMTs is fairly consistent with the general populace's average.  For a different set of blood pressures, however:

In [18]:
EMT_bp2 = [123, 129, 127, 124, 144, 130, 141, 126, 131, 121, 128, 139, 126, 142, 124, 129, 139, 141, 144, 132, 
           122, 121, 130, 123, 115, 130, 123, 124, 129, 129, 124, 134, 139, 124, 131, 121, 133, 140, 137, 142, 
           133, 140, 142, 133, 130, 133, 135, 119, 133, 137]

stats.ttest_1samp(EMT_bp2,mean_sbp)

Ttest_1sampResult(statistic=10.42986165960409, pvalue=4.886680129863528e-14)

The p-value is $\cong 0.000000000000049$, which is much less than the significance level of $0.05$.  Thus, we can reject the null hypothesis that this particular set of EMTs' blood pressure is consistent with the general populace's average blood pressure.

# Type I/II Errors

We've set up our null hypothesis, chosen our test, selected the p-value threshold, run the test, and chosen whether or not to reject the null hypothesis.  It's quite possible that our choice was correct, but it's also possible that it was not.  We can categorize incorrect choices with the following errors:

1. **Type I Error**: the null hypothesis was true, but we rejected it.  
2. **Type II Error**: the null hypothesis was false, but we did not reject it.

One way to think about this is Type I errors are risky, you banked on the existence of something that wasn't there; Type II errors are timid, and you played it safe assuming nothing was there.  We can sum up the possibilities for a given null hypothesis $H_0$ with the following table:

|                     ||      $H_0$ False               |    $H_0$ True                 |
|--------------------:||:------------------------------:|:-----------------------------:|
|     **Reject** $H_0$||     Correct (True Positive)    | Type I Error (False Positive) |
|     **Accept** $H_0$||Type II Error (False Negative) |    Correct (True Negative)    | 

We will be revisiting these when talking about confusion matrices for classification model validation a little later.

When one is testing multiple hypotheses at the same time, the chance of these types of errors increases. One method to control errors is the **Bonferroni Correction**: suppose we have hypotheses $H_1,\ldots,H_n$ with corresponding p-values that we have chosen $p_1,\ldots,p_n$.  If we want to control the probability of making at least one Type I error to be at most $\alpha$, then the Bonferroni Correction tells us to reject all null hypotheses where $p_i \leqslant \frac{\alpha}{n}$.  For example, if we had 10 hypotheses and our desired control level was $0.05$, then we would assess each hypothesis at the significance level of $0.05/10 = 0.005$. (Question:  what issues can arise by doing this?)

## Training/Test Sets

A popular way of validating models is by dividing your data set into two sections (not necessarily of equal size): a **training set** that will be used to create a model, and a **test** set to measure how accurately the model predicts those elements of the data set not included in the modelling procedure.  There are a number of measurements we can use; for a set of $n$ predictions $Z=\{z_1,z_2,\ldots,z_n\}$, and corresponding observations $Y=\{y_1,y_2,\ldots,y_n\}$, we have:

1. **Mean-squared error** measures the average squared-distance between the observations and the model predictions:
$$  \text{MSE} = \frac{1}{n}\sum_{i=1}^n(z_i-y_i)^2 $$
This measure is not in the same units as the data.

2. **Mean absolute error** measures the average distance between the observations and the model predictions: 
$$ \text{MAE} = \frac{1}{n}\sum_{i=1}^n \big|z_i-y_i \big| $$
This measure is in the same units as the data.

3. **Coefficient of determination**, often denoted $R^2$,  
$$ R^2 = 1 - \frac{\sum_{i=1}^n(z_i-y_i)^2}{\sum_{i=1}^n(y_i-\mu_Y)^2}  = 1-\frac{\text{MSE}}{n\cdot{\sigma_y}^2}$$
$R^2$ values close to $1$ can indicate a good fit of an OLS linear regression model, though it does not guarantee it, as Anscombe's Quartet shows:  each model in the quartet have the same $R^2$ value.  Thus, the coefficient of determination should be used in conjunction with other measurements.

Let's re-visit one of our linear regression examples from Unit 8:

In [ ]:
from sklearn import model_selection, metrics

In [ ]:
x = np.arange(0,20,1)
y = [1.3, 2.2, 1.0, 3.1, 2.8, 1.5, 2.1, 2.6, 3.7, 3.5, 3.7, 2.9, 3.7, 4.0, 4.3, 4.7, 4.4, 3.9, 4.8, 4.4]
scatter(x,y);

In [ ]:
df = pd.DataFrame({'Data':y},index=x)
df

For the purposes of example, we will use the following splits in the data:

In [ ]:
tr1 = np.array([13,17,8,18,9,15,14,4,0,1,2,5,16,3,11])
tst1 = np.array([19,12,6,7,10])

In [ ]:
train = df.iloc[tr1]
train

In [ ]:
test = df.iloc[tst1]
test

We can scatter plot the training and test data together:

In [ ]:
scatter(train.index,train['Data'],label='Training Data')
scatter(test.index,test['Data'],color='#00CC00',label='Test Data')
plt.legend();

We run the linear regression on the training set only:

In [ ]:
slope, intercept, r_value, p_value, slope_std_error = stats.linregress(train.index,train['Data'] )

In [ ]:
train_regr = intercept + slope * train.index

scatter(train.index,train['Data'])
plt.plot(train.index,train_regr, color='red');

We compute the $R^2$ and MAE for the model on the *training set*:

In [ ]:
metrics.r2_score(train['Data'], train_regr)

In [ ]:
metrics.mean_absolute_error(train['Data'], train_regr)

Then we compare to the measures for the model on the *test set*:

In [ ]:
metrics.r2_score(test['Data'], intercept + slope*test.index)

In [ ]:
metrics.mean_absolute_error(test['Data'], intercept + slope*test.index)

Both the measures improved on the test data, as $R^2$ got closer to $1$, and the MAE reduced in size.  These are good signs for our model.  Here is the regression line in comparison to the test data:

In [ ]:
train_regr = intercept + slope * train.index

scatter(test.index,test['Data'])
plt.plot(test.index,intercept + slope*test.index, color='red');

And here is the whole data set together, plotted with the regression line:

In [ ]:
scatter(train.index,train['Data'],label='Training Data')
scatter(test.index,test['Data'],color='#00CC00',label='Test Data')
plt.plot(x,intercept + slope*x, color='red',label='Regression Model')
plt.legend();

The metrics for the whole set are:

In [ ]:
print('R-squared value is', metrics.r2_score(y, intercept + slope*x))
print('MAE is', metrics.mean_absolute_error(y, intercept + slope*x))
print('Mean and std are', df['Data'].mean(), df['Data'].std())

The combination of the fairly high $R^2$, fairly low MAE on the scale of the data set, improvement of values from training to test, and a visual inspection of the model's graph show us that this model likely fits the data fairly well.  (One thing to keep in mind is the population size.)

What if we had split it differently?  Since our data set is small, the model validation will be very sensitive to small changes, but for illustrative purposes we will try a random splitting:  we can use scikit-learn to split the data set into two sections.  If we do not specify a size for the test set, it will default to picking $25\%$ of total data set size.

In [ ]:
train1, test1 = model_selection.train_test_split(df)

In [ ]:
len(test1)/(len(train1)+len(test1))

In [ ]:
train1

In [ ]:
test1

We run the linear regression on the training set only:

In [ ]:
slope, intercept, r_value, p_value, slope_std_error = stats.linregress(train1.index,train1['Data'] )

In [ ]:
train_regr = intercept + slope * train1.index

scatter(train1.index,train1['Data'])
plt.plot(train1.index,train_regr, color='red');

We compute the $R^2$ and MAE for the model on the *training set*:

In [ ]:
metrics.r2_score(train1['Data'], train_regr)

In [ ]:
metrics.mean_absolute_error(train1['Data'], train_regr)

Then we compare to the measures for the model on the *test set*:

In [ ]:
metrics.r2_score(test1['Data'], intercept + slope*test1.index)

In [ ]:
metrics.mean_absolute_error(test1['Data'], intercept + slope*test1.index)

How do the training/test set metrics compare?  What can we conclude in this situation?

Again, let us look at the test set against the model, and then the entire data set:

In [ ]:
train_regr = intercept + slope * train1.index

scatter(test1.index,test1['Data'])
plt.plot(test1.index,intercept + slope*test1.index, color='red');

And here is the whole data set together, plotted with the regression line:

In [ ]:
scatter(train1.index,train1['Data'],label='Training Data')
scatter(test1.index,test1['Data'],color='#00CC00',label='Test Data')
plt.plot(x,intercept + slope*x, color='red',label='Regression Model')
plt.legend();

The metrics for the whole set are:

In [ ]:
print('R-squared value is', metrics.r2_score(y, intercept + slope*x))
print('MAE is', metrics.mean_absolute_error(y, intercept + slope*x))
print('Mean and std are', df['Data'].mean(), df['Data'].std())

Based on these tests, how confident are we in our model?

As a final example, let us recall briefly the quadratic data set from Unit 8 that we made a (poor) linear regression model for:

In [ ]:
x_q = np.array([0,0.9,1.5,2.1,2.4,2.9,3.1,3.8,4.4,5.5])
y_q = np.array([-7.3, -2.1, -0.45, 1.39, 1.14, 1.92, 2.05, 1.26, 0.09, -4.35])
x_r = np.arange(0,6,0.01)

In [ ]:
int_q, sl_q = np.polynomial.polynomial.polyfit(x_q, y_q, 1)
plt.plot(x_q,int_q + sl_q*x_q , color='green');
scatter(x_q,y_q);

Inspection of the graph, combined with the very low $R^2$ score

In [ ]:
 metrics.r2_score((x_q,y_q), (x_q,intercept + slope*x_q))

helps convince us that this model is probably not very accurate for the data set.

### Classification Models

We can apply the same techniques to validating classification models as seen in the previous unit.  In this case, we need to choose an appropriate metric to validate; while there are a number of choices, one of the most common is the accuracy of prediction.  First, let us pull in a data set and model we've seen:

In [19]:
an = pd.read_csv('animals.csv',index_col=0)
an

,Legs,Cold or Warm Blooded,Covering,Aquatic,Aerial,Lays Eggs,Category
Tree Frog,4,cold,none,partial,no,yes,amphibian
Cane Toad,4,cold,none,no,no,yes,amphibian
Ball Python,0,cold,scales,no,no,yes,reptile
Dove,2,warm,feathers,no,yes,yes,bird
Human,2,warm,hair,no,no,no,mammal
Lion,4,warm,fur,no,no,no,mammal
Chihuaha,4,warm,fur,no,no,no,mammal
Crow,2,warm,feathers,no,yes,yes,bird
Monitor Lizard,4,cold,none,no,no,yes,reptile
Veiled Chameleon,4,cold,scales,no,no,yes,reptile


We'll create two partitions of our data:

In [20]:
train_c = an.iloc[[1,2,4,5,6,7,9,10,11,12,13,14,15,16,18,21,22,23]] 
test_c = an.iloc[[0,3,8,17,19,20]]

In [21]:
train_c

,Legs,Cold or Warm Blooded,Covering,Aquatic,Aerial,Lays Eggs,Category
Cane Toad,4,cold,none,no,no,yes,amphibian
Ball Python,0,cold,scales,no,no,yes,reptile
Human,2,warm,hair,no,no,no,mammal
Lion,4,warm,fur,no,no,no,mammal
Chihuaha,4,warm,fur,no,no,no,mammal
Crow,2,warm,feathers,no,yes,yes,bird
Veiled Chameleon,4,cold,scales,no,no,yes,reptile
Humpback Whale,0,warm,hair,yes,no,no,mammal
Eastern Newt,4,cold,none,partial,no,yes,amphibian
Falcon,2,warm,feathers,no,yes,yes,bird


In [22]:
test_c

,Legs,Cold or Warm Blooded,Covering,Aquatic,Aerial,Lays Eggs,Category
Tree Frog,4,cold,none,partial,no,yes,amphibian
Dove,2,warm,feathers,no,yes,yes,bird
Monitor Lizard,4,cold,none,no,no,yes,reptile
Red back Salamander,4,cold,none,partial,no,yes,amphibian
Parrot,2,warm,feathers,no,yes,yes,bird
Rabbit,4,warm,fur,no,no,no,mammal


Now we will use the LabelEncoder on each, and create a model on the training set.

In [23]:
from sklearn import model_selection, metrics, linear_model, datasets, feature_selection, tree
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

d = defaultdict(LabelEncoder)

animals = tree.DecisionTreeClassifier()
le = LabelEncoder()
tn1 = (train_c[['Cold or Warm Blooded','Covering','Aquatic','Aerial','Lays Eggs']]).apply(lambda x: d[x.name].fit_transform(x))
train_encoded = pd.concat([train_c['Legs'],tn1],axis=1)

train_encoded

,Legs,Cold or Warm Blooded,Covering,Aquatic,Aerial,Lays Eggs
Cane Toad,4,0,3,0,0,1
Ball Python,0,0,4,0,0,1
Human,2,1,2,0,0,0
Lion,4,1,1,0,0,0
Chihuaha,4,1,1,0,0,0
Crow,2,1,0,0,1,1
Veiled Chameleon,4,0,4,0,0,1
Humpback Whale,0,1,2,2,0,0
Eastern Newt,4,0,3,1,0,1
Falcon,2,1,0,0,1,1


In [31]:
ts1 = (test_c[['Cold or Warm Blooded','Covering','Aquatic','Aerial','Lays Eggs']]).apply(lambda x: d[x.name].fit_transform(x))
test_encoded = pd.concat([test_c['Legs'],ts1],axis=1)

test_encoded

,Legs,Cold or Warm Blooded,Covering,Aquatic,Aerial,Lays Eggs
Tree Frog,4,0,2,1,0,1
Dove,2,1,0,0,1,1
Monitor Lizard,4,0,2,0,0,1
Red back Salamander,4,0,2,1,0,1
Parrot,2,1,0,0,1,1
Rabbit,4,1,1,0,0,0


In [32]:
animals.fit(train_encoded, train_c['Category'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

We can now compute the accuracy of the model's class prediction on the training set.  We imported sklearn's `metrics` package, and will be using the `accuracy_score` metric:

In [33]:
# We use the model to predict the test_encoded data
test_pred_class = animals.predict(test_encoded)

print(metrics.accuracy_score(test_c['Category'], test_pred_class))

0.8333333333333334


So the prediction accuracy of our model was $\cong 83.3\%$, which is fairly high.  We can compare the predicted vs actual categories to see where the issue was:

In [35]:
test_pred_class

array(['amphibian', 'bird', 'amphibian', 'amphibian', 'bird', 'mammal'],
      dtype=object)

In [36]:
test_c['Category']

Tree Frog              amphibian
Dove                        bird
Monitor Lizard           reptile
Red back Salamander    amphibian
Parrot                      bird
Rabbit                    mammal
Name: Category, dtype: object

One way we can visualize the results is with a *confusion matrix*.  When we have $n$ classes, a confusion matrix is an $n\times n$ matrix, where the element in row $i$ column $j$ refers to the number of observations predicted to be in the class corresponding to $j$, but known to actually be in the class corresponding to $i$.  We can compute our confusion matrix via:

In [46]:
conf_matrix = metrics.confusion_matrix(test_c['Category'], test_pred_class,labels=['amphibian','bird','mammal','reptile'])
conf_matrix

array([[2, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]])

In other words:

|                       |  Predicted Amphibian |  Predicted Bird | Predicted Mammal  | Predicted Reptile  |
|---------------------:|:---------------------:|:---------------:|-------------------|--------------------|
| **Actual Amphibian** |           2           |       0         |    0              |    0              |
| **Actual Bird**      |            0          |        2        |    0              |    0              |
| **Actual Mammal**    |             0         |         0       |    1              |    0              |
| **Actual Reptile**   |              1        |          0      |    0              |    0              |

There were two actual amphibians, and two correctly predicted amphibians.  There was one actual reptile, but it was incorrectly predicted to be a bird; there were 0 predicted actual reptiles.  If we look at confusion tables for individual classes, we get a visualization of Type I/II errors: 

|                      |  Predicted Reptile |  Predicted Non-Reptile | 
|---------------------:|:---------------------:|:---------------:|
| **Actual Reptile **         |           0           |       1         |
| **Actual Non-Reptile**      |            0          |        5        |

There were 5 non-reptiles corrected predicted to be non-reptiles (true negatives), but there was 1 reptile incorrectly predicted to be a non-reptile (a false-negative).  

## Cross-Validation

For our validation technique, we split the data set into two sections to train and test.  Instead of just doing this once, we could do this multiple times, for different selections of testing and training data.  While there are a number of ways to do this, we will discuss **k-fold cross validation**.  In k-fold cross-validation, the origianl data set is divided randomly into $k$-many subsets of equal size.  For the first iteration of the validation, the first subset is set aside for testing, while the other $k-1$ subsets are used to train the data.  This process is repeated, changing which subset is the test set, until all possible combinations are exhausted.  

<img src="http://i.imgur.com/pMepZ3V.jpg",width=550,height=550>

We can use the `model_selection.KFold()` function in scikit-learn to create splits for us.  We define a variable name to be `model_selection.KFold()` with arguments `n_splits=` the number of splits we want, and `shuffle=True` if we want it to randomize the order of the index before splitting.  Doing this to our data set `df` from our linear regression model data from above:

In [52]:
kf = model_selection.KFold(n_splits=4,shuffle=True)

We can then tell this `kf` instance to split our data frame into training and test data:

In [53]:
for train, test in kf.split(df):
    print(train,test)

NameError: name 'df' is not defined

Now that we have four sets of training and test data, we could run the test above on each set to compare the $R^2$ and $MAE$, as well as the visualization of the models to see how well our linear regression model works.  We can set variables to each of the splits:

In [49]:
set1,set2,set3,set4 = kf.split(df)

NameError: name 'df' is not defined

In [41]:
set1

NameError: name 'set1' is not defined

In [42]:
df_train1 = df.iloc[set1[0],:]
df_train1

NameError: name 'df' is not defined

In [ ]:
df_test1 = df.iloc[set1[1],:]
df_test1

(Note that we use `.iloc[]` here because the split gives us *integer* indices.)  Now we can proceed by modelling and comparing each of the training/test sets, and collecting all of the metrics together.

### Classification Models

Cross validation can also be used for classification models, using the appropriate metrics for comparing training/test sets.  For example, we could do a 3-fold cross validation on the animals set, splitting:

In [50]:
kf3 = model_selection.KFold(n_splits=3,shuffle=True)
for train, test in kf3.split(an):
    print(train,test)

[ 4  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23] [0 1 2 3 5 6 7 8]
[ 0  1  2  3  5  6  7  8 10 15 16 17 18 21 22 23] [ 4  9 11 12 13 14 19 20]
[ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 19 20] [10 15 16 17 18 21 22 23]


In [51]:
an_set1,an_set2,an_set3 = kf3.split(an)

# Training 1
an.iloc[set1[0]]

NameError: name 'set1' is not defined

In [45]:
# Test 1
an.iloc[set1[1]]

NameError: name 'set1' is not defined

With classification models, however, problems can arise when randomly splitting into training/test.  What is one major problem can occur that would almost completely invalidate a given training/test set?

# Assignment 9

1. Consider the data set <br>
`dfcl = pd.DataFrame({'CL Price': [35, 63, 48, 46, 66, 65, 35, 65, 50, 68, 43, 57, 39, 35, 68, 95, 56, 50, 42, 38, 47, 42, 39, 44, 52, 68, 49, 34, 39, 59, 60, 60, 60, 63, 58, 38, 37, 33, 36, 64, 38, 51, 60, 62, 65, 43, 38, 41, 48, 61]}`.<br>
Are there any values more than 3 standard deviations away from the mean? If so, can we conclude they are outliers?
2.  1.  You flip a coin $n$ times and it comes up heads on each flip. Use the hypothesis testing framework to create a statistical test that will allow you to decide if the coin is biased. How big does $n$ need to be in order to decide that the evidence is significant evidence that the coin is biased? (The ambiguity here is intentional: what does it mean for evidence to be significant?)
   2.   You flip another coin $n$ times and it comes up tails 48% of the time. Use the hypothesis testing to create a statistical test that will allow you to decide if the coin is biased. How big does $n$ need to be for significance? [<em>Important hints for this one</em>. Computing exact p-values like we did in question 1 will probably be infeasible here (why?), so we'll need to use a different method. Notice that we can compute the proportion of tails that come up by encoding the flips with a 1 for tails and 0 for heads, and taking the <em>mean</em> of that. Since the proportion of tails is computed as a <em>sample mean</em>, we get to use the central limit theorem, which says that the proportion of tails (approximately) follows a normal distribution with mean equal to the true mean, and standard deviation equal to the true standard deviation divided by $\sqrt{n}$].
   3.  Based on your answers from 1 and 2, what can you say about the relationship between sample size and statistical significance?

3.  Recall the vaguely quadratic data set from the previous Unit:<br>
`x_q = np.array([0,0.9,1.5,2.1,2.4,2.9,3.1,3.8,4.4,5.5])
y_q = np.array([-7.3, -2.1, -0.45, 1.39, 1.14, 1.92, 2.05, 1.26, 0.09, -4.35])`<br>
Create a training set and a test set from this data by removing two (non-endpoint) points for testing.  Train a quadratic regression model, a linear regression model, and a spline interpolation model on the training set.  Determine which model most accurately predicts the test set.

4. Using the 4-fold data splitting on `df` described in the last section to perform a 4-fold cross validation of the `stats.linregress` linear regression modelling for `df`.  Compare $R^2$, $MAE$, and any other measures you wish.  Don't forget to plot each of the models.

5.  Adapt the following code to the animals model to visualize the confusion matrix we created:<br> http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
